In [1]:
import math
from datetime import datetime
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GlobalAveragePooling2D, Input, Dense, Activation
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, accuracy_score, recall_score, precision_score, f1_score
import random as python_random

#pip install image-classifiers==1.0.0b1
from classification_models.tfkeras import Classifiers

In [2]:

np.random.seed(2021)
python_random.seed(2021)
tf.random.set_seed(2021)

In [3]:
mirrored_strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(mirrored_strategy.num_replicas_in_sync))

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of devices: 4
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPUs will likely run quickly with dtype policy mixed_float16 as they all have compute capability of at least 7.0
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale


In [4]:
data_df = pd.read_csv('modified_viewposition_race_4-race-ethnicity_60-10-30_split_with_gender_age_ver_b.csv')
data_df = data_df[data_df.race.isin(['WHITE','BLACK/AFRICAN AMERICAN','ASIAN'])]

In [5]:
data_df.split.value_counts(normalize=True)

train       0.598783
test        0.301700
validate    0.099517
Name: split, dtype: float64

In [6]:
data_df.race.value_counts(normalize=True)

WHITE                     0.775148
BLACK/AFRICAN AMERICAN    0.186701
ASIAN                     0.038151
Name: race, dtype: float64

In [7]:
data_df.subject_id = data_df.subject_id.astype(str)
data_df.study_id = data_df.study_id.astype(str)
data_df = data_df.fillna(0)
data_df.insert(2, "path", "")
data_df.path = data_df.subject_id.str[0:2]
data_df.path = "p" + data_df.path
data_df.path = data_df.path + "/p" + data_df.subject_id + "/s" + data_df.study_id + "/" + data_df.dicom_id + ".jpg"

In [8]:
train_df = data_df[data_df.split=="train"]
validation_df = data_df[data_df.split=="validate"]
test_df = data_df[data_df.split=="test"]

In [9]:
#False indicates no patient_id shared between groups

unique_train_id = train_df.subject_id.unique()
unique_validation_id = validation_df.subject_id.unique()
unique_test_id = test_df.subject_id.unique()
all_id = np.concatenate((unique_train_id, unique_validation_id, unique_test_id), axis=None)

def contains_duplicates(X):
    return len(np.unique(X)) != len(X)

contains_duplicates(all_id)

False

In [10]:
HEIGHT, WIDTH = 320, 320

arc_name = "MIMIC-" + str(HEIGHT) + "x" + str(WIDTH) + "60-10-30-split-resnet-Float16_3-race_detection"

In [11]:
resnet34, preprocess_input = Classifiers.get('resnet34')

In [12]:
with mirrored_strategy.scope():
    input_a = Input(shape=(HEIGHT, WIDTH, 3))
    base_model = resnet34(input_tensor=input_a, include_top=False, input_shape=(HEIGHT,WIDTH,3), weights='imagenet')
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(3, name='dense_logits')(x)
    output = Activation('softmax', dtype='float32', name='predictions')(x)
    model = Model(inputs=[input_a], outputs=[output])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [13]:
learning_rate = 1e-3
momentum_val=0.9
decay_val= 0.0
batch_s = 256 # may need to reduce batch size if OOM error occurs
train_batch_size = batch_s
test_batch_size = 256

reduce_lr = ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.1, patience=2, min_lr=1e-5, verbose=1)

adam_opt = tf.keras.optimizers.Adam(learning_rate=learning_rate, decay=decay_val)
adam_opt = tf.keras.mixed_precision.LossScaleOptimizer(adam_opt)


with mirrored_strategy.scope():
    model.compile(optimizer=adam_opt,
                    loss=tf.losses.CategoricalCrossentropy(),
                    metrics=[
                        tf.keras.metrics.AUC(curve='ROC', name='ROC-AUC'),
                        tf.keras.metrics.AUC(curve='PR', name='PR-AUC')
                    ],
    )

In [14]:
train_gen = ImageDataGenerator(
            rotation_range=15,
            fill_mode='constant',
            horizontal_flip=True,
            zoom_range=0.1,
            preprocessing_function=preprocess_input
            )

validate_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [15]:
train_batches = train_gen.flow_from_dataframe(train_df, directory="/tf/notebooks/SSD_data/mimic_directory/files", x_col="path", y_col="race", class_mode="categorical",target_size=(HEIGHT, WIDTH),shuffle=True,seed=2021,batch_size=train_batch_size, dtype='float32')
validate_batches = validate_gen.flow_from_dataframe(validation_df, directory="/tf/notebooks/SSD_data/mimic_directory/files", x_col="path", y_col="race", class_mode="categorical",target_size=(HEIGHT, WIDTH),shuffle=False,batch_size=test_batch_size, dtype='float32')        

Found 116331 validated image filenames belonging to 3 classes.
Found 19334 validated image filenames belonging to 3 classes.


In [16]:
train_epoch = math.ceil(len(train_df) / train_batch_size)
val_epoch = math.ceil(len(validation_df) / test_batch_size)

In [17]:
var_date = datetime.now().strftime("%Y%m%d-%H%M%S")
ES = EarlyStopping(monitor='val_loss', mode='min', patience=4, restore_best_weights=True)
checkloss = ModelCheckpoint("../saved_models/racial_bias/trials/" + str(arc_name) + "_CXR_LR-" + str(learning_rate) + "_" + var_date+"_epoch:{epoch:03d}_val_loss:{val_loss:.5f}.hdf5", monitor='val_loss', mode='min', verbose=1, save_best_only=True, save_weights_only=False)

In [18]:
model.fit(train_batches,
            validation_data=validate_batches,
            epochs=100,
            steps_per_epoch=int(train_epoch),
            validation_steps=int(val_epoch),
            workers=32,
            max_queue_size=50,
            shuffle=True,
            callbacks=[checkloss, reduce_lr, ES]
           )

Epoch 1/100
INFO:tensorflow:batch_all_reduce: 108 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 108 all-reduces with algorithm = nccl, num_packs = 1
455/455 [==============================] - 756s 1s/step - loss: 0.3512 - ROC-AUC: 0.9656 - PR-AUC: 0.9394 - val_loss: 0.3471 - val_ROC-AUC: 0.9672 - val_PR-AUC: 0.9414

Epoch 00001: val_loss improved from inf to 0.34711, saving model to ../saved_models/racial_bias/trials/MIMIC-320x32060-10-30-split-resnet-Float16_3-race_detection_CXR_LR-0.001_20210628-011052_epoch:001_val_loss:0.34711.hdf5


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/100
455/455 [==============================] - 724s 2s/step - loss: 0.2392 - ROC-AUC: 0.9831 - PR-AUC: 0.9695 - val_loss: 0.4755 - val_ROC-AUC: 0.9425 - val_PR-AUC: 0.8959

Epoch 00002: val_loss did not improve from 0.34711
Epoch 3/100
455/455 [==============================] - 724s 2s/step - loss: 0.2044 - ROC-AUC: 0.9874 - PR-AUC: 0.9770 - val_loss: 0.2485 - val_ROC-AUC: 0.9820 - val_PR-AUC: 0.9680

Epoch 00003: val_loss improved from 0.34711 to 0.24853, saving model to ../saved_models/racial_bias/trials/MIMIC-320x32060-10-30-split-resnet-Float16_3-race_detection_CXR_LR-0.001_20210628-011052_epoch:003_val_loss:0.24853.hdf5
Epoch 4/100
455/455 [==============================] - 729s 2s/step - loss: 0.1828 - ROC-AUC: 0.9896 - PR-AUC: 0.9810 - val_loss: 0.3081 - val_ROC-AUC: 0.9735 - val_PR-AUC: 0.9520

Epoch 00004: val_loss did not improve from 0.24853
Epoch 5/100
455/455 [==============================] - 727s 2s/step - loss: 0.1670 - ROC-AUC: 0.9912 - PR-AUC: 0.9839 - val_los

In [19]:
test_df.race.value_counts(normalize=True)

WHITE                     0.780650
BLACK/AFRICAN AMERICAN    0.181475
ASIAN                     0.037875
Name: race, dtype: float64

In [20]:
test_batch_size = 256

In [22]:
test_batches = validate_gen.flow_from_dataframe(test_df, directory="/tf/notebooks/SSD_data/mimic_directory/files", x_col="path", y_col="race", class_mode="categorical",target_size=(HEIGHT, WIDTH),shuffle=False,batch_size=test_batch_size, dtype='float32')        

Found 58614 validated image filenames belonging to 3 classes.


In [23]:
with mirrored_strategy.scope():

    multilabel_predict_test = model.predict(test_batches, max_queue_size=10, verbose=1, steps=math.ceil(len(test_df)/test_batch_size), workers=16)


229/229 [==============================] - 343s 1s/step


In [24]:
input_prediction = multilabel_predict_test
input_df = test_df
input_prediction_df = pd.DataFrame(input_prediction)
true_logits = pd.DataFrame()
loss_log = pd.DataFrame()

In [25]:
def stat_calc(input_prediction_df, input_df):
    ground_truth = input_df.race
    pathology_array=[
        'ASIAN',
        'BLACK/AFRICAN AMERICAN',
        'WHITE'
        ]
    i=0
    auc_array = []
    for pathology in pathology_array:
    
        new_truth = (ground_truth.str.contains(pathology)).apply(int)
        input_prediction_val = input_prediction_df[i]
        val = input_prediction_val
        AUC = roc_auc_score(new_truth, val)
        true_logits.insert(i, i, new_truth, True)
        auc_array.append(AUC)
        i += 1
        
    progress_df = pd.DataFrame({'Study':pathology_array, 'AUC':auc_array})
    print(progress_df)

In [26]:
stat_calc(input_prediction_df, input_df)

                    Study       AUC
0                   ASIAN  0.979197
1  BLACK/AFRICAN AMERICAN  0.974881
2                   WHITE  0.973417
